# Chapter 1

## Creating arbitrary chunks

First, we conditionally import the `pyactr` module as `actr`.

In [1]:
import pyactr as actr

Let's first declare a chunk type to model the lexicon. The `chunktype` function creates a type `word` with some slots, seperated by commas.

In [2]:
actr.chunktype("word", "form, meaning, category, number")

Now that we've declared a chunk type, we can create chunks of that type.

Note that the function `makechunk` only has two obligatory arguments:
 - `typename`
 - `nameofchunk`
 
 Unspecified slots are left empty.

In [3]:
carLexeme = actr.makechunk(nameofchunk="car1",
                          typename="word",
                          form="car",
                          meaning="[[car]]",
                          category="noun",
                          number="sg")

print(carLexeme)

word(category= noun, form= car, meaning= [[car]], number= sg)


N.b. that chunks can be extended with new slots not originally present in the chunk type declaration.

In [4]:
carLexeme2 = actr.makechunk(nameofchunk="car1",
                          typename="word",
                          form="car",
                          meaning="[[car]]",
                          category="noun",
                          number="sg",
                           synfunction="subj")


/nix/store/x26x643i7wrljvcd2c9zkdd89f4mvzxa-python3-3.7.7-env/lib/python3.7/site-packages/pyactr/chunks.py:130: UserWarning: Chunk type word is extended with new attributes
  warnings.warn("Chunk type %s is extended with new attributes" % typename)


The `chunkstring` function is an alternative method for declaring a new chunk:

(n.b. triple quotation in python indicates that a string can appear on more than one line.)

In [5]:
carLexeme3 = actr.chunkstring(string="""
    isa word
    form car
    meaning '[[car]]'
    category noun
    number sg
    synfunction subj
""")

print(carLexeme3)

word(category= noun, form= car, meaning= [[car]], number= sg, synfunction= subj)


Treating chunks as attribute value sets induces natural notions of equality and implication. The standard python comparison operators are overloaded to reflect this.

N.b. the chunk type is completely ignored for the purpose of these comparisons; the chunk type is just syntactic sugar for human modellers, it has no status in the theory of ACT-R.

In [6]:
carLexeme2 == carLexeme3

True

In [7]:
carLexeme == carLexeme2

False

In [8]:
carLexeme < carLexeme2

True

## Beyond arbitrary chunks

We're going to build an extremely simple mind that can do subject agreement. First, let's build a container for a mind (a model).

In [9]:
agreement = actr.ACTRModel ()

For convenience, any model comes with a declarative memory module with an empty retrieval buffer. Modules are attributes of models.

In [10]:
agreement.decmem
agreement.goal
agreement.retrieval

set()

Let's declare a shorter alias for the declarative memory module.

In [11]:
dm = agreement.decmem

We can invoke the `add` method associated with the declarative memory module in order to add chunks to declarative memory.

In [12]:
dm.add(carLexeme2)
print(dm)

{word(category= noun, form= car, meaning= [[car]], number= sg, synfunction= subj): array([0.])}


Note that when we inspect `dm`, we see the chunk we just added, alongside the chunk encoding time. Since we haven't run the model yet, this time is 0.

In order to model subject-verb agreement, we're going to be making some simplifying assumptions. We won't say anything about how parsing works, just assume that declarative memory already contains the suvject  of the clause, and the current verb is present in the goal buffer.

We're going to need three productions in procedural memory:
- if the goal has a verb chunk, and the current task is to agree, then retrieve the subject.
- If the number of the subject is `=x`, then the number of the verb in the goal should also be `=x`.
- If the verb is assigned a number, the task is done.

### Noun retrieval

Productions are created by the method `productionstring`.

The `==>` acts as a seperator between the preconditions and the actions.

Preconditions are specified for two buffers.
- `=g>` indicates the target buffer (`g`: the *goal* buffer), and the type of precondition this buffer has to satisfy (`=`: subsumption; this is highly confusing). So, the chunk currently stored in the goal buffer must be subsumed by the chunk characterised on the following lines. Since the precondition is stated as subsumption, the chunk in the goal buffer could have other slot value pairs (how do we square this with subsumption?).
- `?retrieval>` checks whether the retrieval buffer is in a certain state. The `?` symbol indicates that we're interested in the state of the buffer, not the chunk inside of it. The buffer must be empty for this precondition to be satisfied.

In general we can use `?` to submit a variety of queries regarding the state of buffers.

If the preconditions are both met, two actions are triggered.
-  The current task of the `goal_lexeme` chunk is changed to `trigger_agreement`. This isn' reflected in the book, but it looks like this is really the only thing that needs to be specified.
- The second action is to add a new chunk to the retrieval buffer, this is what the `+` symbol indicates. What gets added to the retrieval buffer is a noun that is the subject of the sentence.

In [13]:
actr.chunktype("goal_lexeme","category, task")

agreement.productionstring(name="retrieve", string="""
    =g>
    isa goal_lexeme
    category verb
    task agree
    ?retrieval>
    buffer empty
    ==>
    =g>
    isa goal_lexeme
    task trigger_agreement
    category verb
    +retrieval>
    isa word
    category noun
    synfunction subject
    """)

{'=g': goal_lexeme(category= verb, task= agree), '?retrieval': {'buffer': 'empty'}}
==>
{'=g': goal_lexeme(category= verb, task= trigger_agreement), '+retrieval': word(category= noun, form= , meaning= , number= , synfunction= subject)}

### Agreement

We now define a second production rule to actually perform the agreement. This comes with two preconditions that check that we are in the correct state:
- The chunk in the goal buffer is subsumed by the chunk described here.
- The chunk in the retrieval buffer is subsumed by the chunk described here.

If we're in the correct state, the action is triggered:
- the chunk in the goal buffer is *updated* such that a new number specification is added, which matches the one on the subject noun we retrieved from declarative memory.
- The task is marked as done.

In [15]:
agreement.productionstring(name="agree", string="""
    =g>
    isa goal_lexeme
    task trigger_agreement
    category verb
    =retrieval>
    isa word
    category noun
    synfunction subject
    number =x
    ==>
    =g>
    isa goal_lexeme
    category verb
    number =x
    task done
    """)

{'=g': goal_lexeme(category= verb, number= , task= trigger_agreement), '=retrieval': word(category= noun, form= , meaning= , number= =x, synfunction= subject)}
==>
{'=g': goal_lexeme(category= verb, number= =x, task= done)}

### Clean-up

The final production rule mops things up, flushing the goal buffer.

In [16]:
agreement.productionstring(name="done", string="""
=g>
isa goal_lexeme
task done
==>
~g>
""")

{'=g': goal_lexeme(category= , number= , task= done)}
==>
{'~g': None}

## Running the model

To get things off the ground, we need to add a goal lexeme to the goal buffer. Since according to ACT-R, higher cognition is goal driven, nothing will happen in the absence of a goal.

In [17]:
actr.chunktype("goal_lexeme", "task, category, number")

agreement.goal.add(actr.chunkstring(string="""
    isa goal_lexeme
    task agree
    category verb"""))

agreement.goal

{goal_lexeme(category= verb, number= , task= agree)}

In [18]:
agreement_sim = agreement.simulation()

- TODO the following currently doesn't deliver the expected results.

In [19]:
agreement_sim.run()

(0, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0, 'PROCEDURAL', 'RULE SELECTED: retrieve')
(0.05, 'PROCEDURAL', 'RULE FIRED: retrieve')
(0.05, 'g', 'MODIFIED')
(0.05, 'retrieval', 'START RETRIEVAL')
(0.05, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.05, 'PROCEDURAL', 'NO RULE FOUND')
(0.1, 'retrieval', 'RETRIEVED: None')
(0.1, 'PROCEDURAL', 'CONFLICT RESOLUTION')
(0.1, 'PROCEDURAL', 'NO RULE FOUND')


Observe that when we run a simulation, every cognitive step takes 50ms. This is the ACT-R default for an elementary cognitive step.

# More examples

## The counting model

### The task

- Given an initial number $n$, and a final number $m$, increment $n$ until $m$ is reached.